In [24]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
import numpy as np
import gensim
from gensim.models import Word2Vec
from nltk.data import find
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import random

# data = pd.read_csv('./all_cc_jokes.csv', sep = ',', index_col = 0, names = ['type', 'link', 'text'])
data = pd.read_csv('combined_jokes.csv', sep = ',', index_col = 0)
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.Word2Vec.load_word2vec_format(word2vec_sample, binary=False)
tfidfvectorizer = TfidfVectorizer(max_features=2000, stop_words='english')
tfidf_counts = tfidfvectorizer.fit_transform(data['text'].values.astype('U'))

In [25]:
def latent_topic(joke_index, tfidf_counts, tfidfvectorizer, model):
    """
    creates a latent topic for a joke using an existing word2vec model
    does so by doing a weighted avg of word2vec 
    """
    print('\rdoing joke {}'.format(joke_index), end='')
    scores = tfidf_counts[joke_index]
    result = None
    for index in scores.indices:
        curr_word = tfidfvectorizer.get_feature_names()[index]
        try:
            if result is None:
                result = model[curr_word]
            else:
                result += model[curr_word]
        except:
            pass
    return result

def latent_topic2(joke_index, tfidf_counts, tfidfvectorizer, model):
    """
    creates a latent topic for a joke using an existing word2vec model
    does so by doing a weighted avg of word2vec 
    """
    print('\rdoing joke {}'.format(joke_index), end='')
    scores = tfidf_counts[joke_index]
    result = None
    for index in scores.indices:
        curr_word = tfidfvectorizer.get_feature_names()[index]
        try:
            if result is None:
                result = model[curr_word].reshape((1,300))
            else:
                result = np.append(result,model[curr_word], axis=0)
        except:
            pass
    if result is None:
        return None
    mean = np.mean(result, axis=0)
    std = np.std(result, axis=0)
    return np.append(mean-std, mean+std, axis = 0).reshape((600))

In [26]:
generate = False
try:
    all_latent_topics = np.load('./latent_topics2.npy')
    print("successfully loaded latent topics for jokes from file")
except:
    generate = True

if generate:
    print("generating latent topics for jokes file,")
    all_latent_topics = np.zeros((data.shape[0],300))
    for i in range(data.shape[0]):
        result = latent_topic(i, tfidf_counts, tfidfvectorizer, model)
        if result is not None:
            all_latent_topics[i] = result 
    print("\ndone! saved as latent_topics.npy")
    np.save('./latent_topics.npy', all_latent_topics)

generating latent topics for jokes file,
doing joke 17853
done! saved as latent_topics.npy


In [5]:
def find_most_similar(index, latent_topics, jokes):
    curr_topic = latent_topics[index]
    
    ssd = np.sum(np.abs(latent_topics - curr_topic), axis=1)
    index = 0

    while ssd[np.argsort(ssd)[index]] == 0 and index < 8:
        print ("skip, identical")
        print ('index', np.argsort(ssd)[index])
        print ('actual', np.sum(np.square(latent_topics[index] - curr_topic)))
        print ('ssd', ssd[np.argsort(ssd)[index]])
        print (jokes[np.argsort(ssd)[index]])
        index += 1
    print (np.sum(np.square(latent_topics[np.argsort(ssd)[index]] - curr_topic)))
    print(np.sum(np.square(latent_topics - curr_topic), axis=1)[np.argsort(ssd)[index]])
    print (ssd[np.argsort(ssd)[index]])
    print (ssd[np.argsort(ssd)[index+1]])
    print (ssd[np.argsort(ssd)[index-1]])

    return np.argsort(ssd)[index]

# set to random, or you can set index to a number
index=random.randint(0,data.shape[0])
# index=13929
print(index)
print (data['text'][index])
similar_index = find_most_similar(index, all_latent_topics, data['text'])
print(similar_index)
print (data['text'][similar_index])

7569
A couple celebrates their 30th anniversary by visiting fence against which they first made love. The husband says, "Come on, for old times' sake." The wife agrees and they both undress. Afterwards, the husband says, "You're even better than you were 30 years ago." His wife replies, "That fence wasn't electrified 30 years ago!"  
skip, identical
index 7569
actual 3.2504038384e+27
ssd 0.0
A couple celebrates their 30th anniversary by visiting fence against which they first made love. The husband says, "Come on, for old times' sake." The wife agrees and they both undress. Afterwards, the husband says, "You're even better than you were 30 years ago." His wife replies, "That fence wasn't electrified 30 years ago!"  
8.72655737864e+14
8.72655737864e+14
422806400.0
28030250240.0
0.0
7562
Two couples go on vacation together. After a week, they are thoroughly bored.   The men decide that maybe life will take on new meaning if they change partners. They all agree that it's an experiment wor

In [19]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import linear_model

def guess_ratings(indices, ratings, joke_features):
    alpha = 9999999999
#     lasso =linear_model.LinearRegression()
#     lasso = Lasso(alpha=alpha, max_iter = 20000, tol=.1)
    lasso = linear_model.Ridge (alpha = .1)
#     lasso = ElasticNet(alpha=alpha, l1_ratio=0.7, max_iter=20000)
#     lasso

    y_pred_lasso = lasso.fit(joke_features[indices], ratings).predict(joke_features)
    y_pred_lasso -= np.mean(y_pred_lasso)
    y_pred_lasso *= 1.5/np.std(y_pred_lasso)
    y_pred_lasso += 2.5
    y_pred_lasso = y_pred_lasso.round()
    y_pred_lasso[np.where(y_pred_lasso > 5)] = 5
    y_pred_lasso[np.where(y_pred_lasso < 1)] = 1
    return y_pred_lasso


# add a joke's index and then its corresponding rating 
indices = [9539, 9943, 14327, 14328, 13048, 13058, 8698, 8701, 2578, 2598, 5497, 6235, 13886, 0, 15017, 13882, 10817, 13867, 14196, 13860, 13857, 10764, 13876, 10830, 10820, 13894, 13889, 13864, 13893, 14086, 13871, 13874, 12070, 13861, 13829, 13871, 12806, 12916, 14609, 14506, 14232, 13892, 13896, 13052, 13078, 4479, 10665, 3220, 5178, 4971, 10065, 14513, 4851, 14453, 14406, 14427, 221, 1399, 9912, 294, 9192, 130, 9960, 5156, 14276, 10745, 10175, 14434]
ratings = [4.5, 2, 3.3, 4, 5, 2, 3.8, 3, 2, 4, 3.5, 1, 3, 4, 3.5, 3, 0, 1, .5, 0, 2, 1, 0, 4, 4.5, 4.5, 2, 3, 3, 1, 2, 4, 1, 4, 2, 3, 4, 0, 4, 1, 2.1, 1, 2, 0, 0, 0, 5, 3, 4.5, 5, 1, 3.3, 3.4, 3, 3, 1, 0, 0, 0, 3, 2, 3, 1, 3, 2, 2, 2, 5]
indices, ratings = np.asarray(indices), np.asarray(ratings)
guesses = guess_ratings(indices, ratings, all_latent_topics)

blah = -1
# uncomment this to get "good jokes"
# blah = random.randint(-1000, -1)

# uncomment this to get "bad jokes"
# blah = random.randint(0, 1000)

# uncomment this to get a completely random joke
# blah = random.randint(0, 15000)


joke_index = np.argsort(guesses)[blah]
while joke_index in indices:
    blah -= 1
    joke_index = np.argsort(guesses)[blah]
# joke_index = random.randint(0, 15000)
print('index: ', joke_index)
print('rating: ', guesses[joke_index])
print(data['text'][joke_index])

index:  14652
rating:  9.0
Yo' Mama is so nasty, the sign in the front yard says, "Beware of Mama." 


In [29]:
# return rating guestimate given existing ratings
def guess_ratings(indices, ratings, joke_features):
    clf = linear_model.Ridge (alpha = .1)
    y_pred = clf.fit(joke_features[indices], ratings).predict(joke_features)
    y_pred -= np.mean(y_pred)
    y_pred *= 1.5/np.std(y_pred)
    y_pred += 2.5
    y_pred = y_pred.round()
    y_pred[np.where(y_pred > 5)] = 5
    y_pred[np.where(y_pred < 1)] = 1
    return y_pred

# get top five jokes
def get_top_five_jokes(indices, ratings, all_jokes, all_latent_topics):
#     all_jokes = load_jokes()['text']
#     all_latent_topics = load_latent_topics()
#     rating_tuples = list(ratings_dict.items())
#     indices = [int(rating[0]) for rating in rating_tuples]
#     ratings = [int(rating[1]) for rating in rating_tuples]

    indices, ratings = np.asarray(indices), np.asarray(ratings)
    guesses = guess_ratings(indices, ratings, all_latent_topics)

    top_five_idx, top_five_txt, top_five_rating = [], [], []

    joke_rankings = np.argsort(guesses)
    pos = -1
    joke_index = joke_rankings[pos]

    while len(top_five_idx) < 5:
        while joke_index in indices or joke_index in top_five_idx:
            pos -= 1
            joke_index = joke_rankings[pos]

        top_five_idx += [joke_index]
        top_five_txt += [all_jokes[joke_index]]
        top_five_rating += [guesses[joke_index]]

    print(top_five_idx)
    print(top_five_txt)
    print(top_five_rating)
    return top_five_idx, top_five_txt, top_five_rating

In [30]:
indices = [9539, 9943, 14327, 14328, 13048, 13058, 8698, 8701, 2578, 2598, 5497, 6235, 13886, 0, 15017, 13882, 10817, 13867, 14196, 13860, 13857, 10764, 13876, 10830, 10820, 13894, 13889, 13864, 13893, 14086, 13871, 13874, 12070, 13861, 13829, 13871, 12806, 12916, 14609, 14506, 14232, 13892, 13896, 13052, 13078, 4479, 10665, 3220, 5178, 4971, 10065, 14513, 4851, 14453, 14406, 14427, 221, 1399, 9912, 294, 9192, 130, 9960, 5156, 14276, 10745, 10175, 14434]
ratings = [4.5, 2, 3.3, 4, 5, 2, 3.8, 3, 2, 4, 3.5, 1, 3, 4, 3.5, 3, 0, 1, .5, 0, 2, 1, 0, 4, 4.5, 4.5, 2, 3, 3, 1, 2, 4, 1, 4, 2, 3, 4, 0, 4, 1, 2.1, 1, 2, 0, 0, 0, 5, 3, 4.5, 5, 1, 3.3, 3.4, 3, 3, 1, 0, 0, 0, 3, 2, 3, 1, 3, 2, 2, 2, 5]
all_jokes = data['text']
get_top_five_jokes(indices, ratings, all_jokes, all_latent_topics)

[13866, 15860, 16956, 14070, 14075]
['A guy walks into a bar and says to the barman, "Give me six double vodkas."\n \n The barman says, "Wow, you must have had one hell of a day."\n \n "Yeah, I just found out my oldest son is gay."\n \n The next day, the same guy comes into the bar and asks for six more double vodkas. When the bartender asks what\'s wrong, the man says, "I just found out that my youngest son is gay, too!"\n \n On the third day, the guy comes into the bar and orders another six double vodkas. The bartender says, "Jesus! Doesn\'t anybody in your family like women?"\n \n The man downs the first drink and shakes his head, "Yeah, my wife!" ', 'I always knew that I could never be a lawyer because of my inability to pass a bar.', 'Great big polar bear(she says what?) It broke the ice!', 'What do you call a nun who just passed her bar exam? A sister-in-law. ', "A doctor notices a sidewalk stand that says 'brains for sale.' He   goes over to investigate and sees a sign that say

([13866, 15860, 16956, 14070, 14075],
 ['A guy walks into a bar and says to the barman, "Give me six double vodkas."\n \n The barman says, "Wow, you must have had one hell of a day."\n \n "Yeah, I just found out my oldest son is gay."\n \n The next day, the same guy comes into the bar and asks for six more double vodkas. When the bartender asks what\'s wrong, the man says, "I just found out that my youngest son is gay, too!"\n \n On the third day, the guy comes into the bar and orders another six double vodkas. The bartender says, "Jesus! Doesn\'t anybody in your family like women?"\n \n The man downs the first drink and shakes his head, "Yeah, my wife!" ',
  'I always knew that I could never be a lawyer because of my inability to pass a bar.',
  'Great big polar bear(she says what?) It broke the ice!',
  'What do you call a nun who just passed her bar exam? A sister-in-law. ',
  "A doctor notices a sidewalk stand that says 'brains for sale.' He   goes over to investigate and sees a si

In [22]:
guesses.max()

5.0